# Create Functions

In [16]:
# Import: Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import random
import time

# Create: MISC Functions

def init_browser():
    # Chrome driver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

# Left (Used for scraping City)
def left(target, char):
    char_position = int(target.find(char))
    return target[:char_position]

# Mid (Used for scraping State)
def mid(target):
    char_position_1 = int(target.find(",") + 2)
    char_position_2 = int(target.find(",") + 4)
    return target[char_position_1:char_position_2]

# Right (Used for scraping Zip)
def right(target):
    return target[-5:]

# Random Sleep Variable Time (Archiving: Needs more granularity)
# def rand_sleep():
#     time.sleep(random.randint(60,180))

# Random Sleep Variable Time
def rand_sleep():
    time.sleep(round(random.uniform(60,114), 2))

# Create: MAIN Functions
   
# Testing Function: Save website to txt > scrape the .txt file
def scrape_house_info(url):
    
    # Initialize: Browser
    browser = init_browser()
    
    # Visiting: Browser
    # url = "https://www.trulia.com/p/ca/buena-park/7770-adams-way-buena-park-ca-90620--2077861683"
    browser.visit(url)
    
    # Create: Soup obj
    html = browser.html
    soup = bs(html, "html.parser")
    
    try:
        # Create and Save: Store the soup obj into a text file (The purpose is to bypass: Google Image Verification Security)
        # This text file will be overwritten each time this function is called
        f = open("staging_soup.txt", "w")
        f.write(str(soup))
        f.close()
    except:
        pass
       
    # Requirement to bypass Google Image Verification Secuirty
    rand_sleep()
    
    # Closing: Browser
    browser.quit()
    
    ########## Begin Scrape: Scraping the text file (Not the URL)

    try:
        # Create: A soup obj from the .txt file mentioned above
        f = open("staging_soup.txt")
        soup = bs(f, "html.parser")
    except:
        pass
        
    # Create: Target objects (1/2)
    try:
        overview = soup.find_all("div", class_="row mbm miniCol24 xsCol24 smlCol24 mdCol16 lrgCol16")[0].find_all("li")
    except:
        pass
    
    try:
        house_type = overview[0].text.strip()
    except:
        pass

    if (house_type == "Single-Family Home"):
    
        # Create: Target objects (2/2)
        try:
            city_state_zip = soup.find_all("span", class_="h6 typeWeightNormal pts man paXxsHidden maXxsHidden")[0].text.strip()
        except:
            pass

        try:
            street = soup.find_all("div", class_="prm pan man defaultLineHeight")[0].text.strip()
        except:
            pass

        try:
            overview = soup.find_all("div", class_="row mbm miniCol24 xsCol24 smlCol24 mdCol16 lrgCol16")[0].find_all("li")
        except:
            pass

        try:
            estimate = soup.find("div", class_="pan ptxsXxsVisible ptlSmlVisible flexSpan").find("div", class_="man").find("div", class_="mvn").find("span", class_="h3")
        except:
            pass

        # Scrape: The following
        
        # 1. City
        try:
            city = left(city_state_zip,",")
        except:
            pass
            
        # 2. State
        try:
            state = mid(city_state_zip)
        except:
            pass

        # 3. Zip
        try:
            zip_code = right(city_state_zip)
        except:
            pass
            
        # 4. Street name
        try:
            street
        except:
            pass

        # 5. Beds
        try:
            beds = left(overview[1].text.strip()," ")
        except:
            pass

        # 6. Baths
        try:
            baths = left(overview[2].text.strip()," ")
        except:
            pass

        # 7. Sq Ft
        try:
            sq_ft = left(overview[5].text.strip()," ").replace(",","")
        except:
            pass

        # 8. Lot Size
        try:
            lot_size = left(overview[4].text.strip()," ").replace(",","")
        except:
            pass

        # 9. House Type
        try:
            house_type = overview[0].text.strip()
        except:
            pass

        # 10. Year Built
        try:
            year_built = overview[3].text[-4:]
        except:
            pass

        # 11. Trulia Estimate
        try:
            trulia_estimate = left(estimate.text.strip(),"\n").replace("$","").replace(",","")
        except:
            pass

        # 12. URL
        try:
            for i in range(len(soup.find_all("link"))):
                if "".join(soup.find_all("link")[i].get("rel")) == "canonical":
                    url = soup.find_all("link")[i].get("href")

        except:
            pass
        
        # Create: Unique ID
        try:
            unique_id = []
            unique_id.extend((city, state, zip_code,street,beds,baths,sq_ft,lot_size,house_type, year_built, trulia_estimate))
        except:
            pass
            
        ########## End Scrape: Soup obj mentioned above

        # Initialize: Python dictionary (To store the scraped data)
        house_info = {}

        # Add and Clean: Scraped data to dict
        try:
            house_info = {
                # "house_no": [int(counter)],
                "unique_id": str("".join(unique_id)),
                "url": str(url),

                "city": str(city),
                "state": str(state),
                "zip": str(zip_code),
                "street": str(street),

                "beds": int(beds),
                "baths": float(baths),
                "sq_ft": int(sq_ft
                             
                "lot_size": int(lot_size),
                "house_type": str(house_type),
                "year_built": int(year_built),

                "trulia_estimate": str(trulia_estimate)
            }

        except:
            pass
        
        # Read-in: Database
        database_house_info = pd.read_csv("house_data.csv")

        # Countif: Counts how many times house_info[unique_id] (dict) is in the database_house_info[unique_id] (df) 
        try:
            countif = database_house_info[database_house_info["unique_id"] == "".join(house_info["unique_id"])].count()["unique_id"]
            countif
        except:
            pass

        # Detecting Duplicates: If Countif >= 1 do not add to db
        try:
            if countif == 0:

                # Append: Dataframe to Database
                database_house_info = database_house_info.append(house_info, ignore_index=True)

                # Save: Info to Dataframe
                database_house_info.to_csv("house_data.csv", index=False, columns=["unique_id",
                                                                                   "url",
                                                                                   "city",
                                                                                   "state",
                                                                                   "zip",
                                                                                   "street",
                                                                                   "beds",
                                                                                   "baths",
                                                                                   "sq_ft",
                                                                                   "lot_size",
                                                                                   "house_type",
                                                                                   "year_built",
                                                                                   "trulia_estimate"
                                                                                  ])

        except:
            pass
        
        else:
            pass

# Read in URL Links

In [17]:
# Create: A DF to read in the "staging_address.csv"
df_staging = pd.read_csv("staging_address.csv")

# Initialize: List
address_url_list = []

# Scrape: Address + append to address_url_list = []
for i in range(len(df_staging)):
    address_url_list.append(df_staging["url_addr"][i])

for i in range(len(address_url_list)):
    print(f'Index Position: {i} | URL: {address_url_list[i]} | File No: {df_staging["file_no"][0]}')

Index Position: 0 | URL: https://www.trulia.com/p/ca/irvine/36-silver-cres-irvine-ca-92603--2080997290 | File No: 1 of 4
Index Position: 1 | URL: https://www.trulia.com/p/ca/irvine/37-silver-cres-irvine-ca-92603--2080997343 | File No: 1 of 4
Index Position: 2 | URL: https://www.trulia.com/p/ca/irvine/38-silver-cres-irvine-ca-92603--2080997387 | File No: 1 of 4
Index Position: 3 | URL: https://www.trulia.com/p/ca/irvine/39-silver-cres-irvine-ca-92603--2080997435 | File No: 1 of 4
Index Position: 4 | URL: https://www.trulia.com/p/ca/irvine/40-silver-cres-irvine-ca-92603--2080997590 | File No: 1 of 4
Index Position: 5 | URL: https://www.trulia.com/p/ca/irvine/41-silver-cres-irvine-ca-92603--2080997648 | File No: 1 of 4
Index Position: 6 | URL: https://www.trulia.com/p/ca/irvine/42-silver-cres-irvine-ca-92603--2080997679 | File No: 1 of 4
Index Position: 7 | URL: https://www.trulia.com/p/ca/irvine/43-silver-cres-irvine-ca-92603--2080997718 | File No: 1 of 4
Index Position: 8 | URL: https:/

# Execute Main Function via For Loop

In [18]:
# This code is: LIVE

counter = 0

for i in range(len(address_url_list)):
    scrape_house_info(address_url_list[i])
    counter = counter + 1
    print("Scrape no: " + str(counter) + " completed. | URL processed: " + str(address_url_list[i]) + " | " + str(len(address_url_list) - 2) + " remaining.")

print("Scrape Batch Completed")

Scrape no: 1 completed. | URL processed: https://www.trulia.com/p/ca/irvine/36-silver-cres-irvine-ca-92603--2080997290 | 828 remaining.
Scrape no: 2 completed. | URL processed: https://www.trulia.com/p/ca/irvine/37-silver-cres-irvine-ca-92603--2080997343 | 828 remaining.
Scrape no: 3 completed. | URL processed: https://www.trulia.com/p/ca/irvine/38-silver-cres-irvine-ca-92603--2080997387 | 828 remaining.
Scrape no: 4 completed. | URL processed: https://www.trulia.com/p/ca/irvine/39-silver-cres-irvine-ca-92603--2080997435 | 828 remaining.
Scrape no: 5 completed. | URL processed: https://www.trulia.com/p/ca/irvine/40-silver-cres-irvine-ca-92603--2080997590 | 828 remaining.
Scrape no: 6 completed. | URL processed: https://www.trulia.com/p/ca/irvine/41-silver-cres-irvine-ca-92603--2080997648 | 828 remaining.
Scrape no: 7 completed. | URL processed: https://www.trulia.com/p/ca/irvine/42-silver-cres-irvine-ca-92603--2080997679 | 828 remaining.
Scrape no: 8 completed. | URL processed: https:/

UnboundLocalError: local variable 'house_type' referenced before assignment